In [ ]:
api_key = ""  #replace it with the openAI api key
load_trained_data = 0
run_openAI_experiments = 1-load_trained_data
# load_prompts_from_Git = 1
# run_openAI_experiments = 0  #openAI experiments
# load_RecLLMparsed_df = load_prompts_from_Git
# load_RecLLMparsed_with_itemIds = load_prompts_from_Git

## Utility Functions

In [ ]:
def print_statistics(data):
    total_interactions = data.shape[0]
    unique_users = data['user'].nunique()
    unique_artists = data['artist'].nunique()
    unique_tracks = data['track_name'].nunique()

    print("\nFinal Dataset Overview:")
    print(f"Total Interactions: {total_interactions}")
    print(f"Unique Users: {unique_users}")
    print(f"Unique Artists: {unique_artists}")
    print(f"Unique Tracks: {unique_tracks}")
    print(f"Average Interactions per User: {total_interactions / unique_users:.2f}")
    print(f"Average Interactions per Track: {total_interactions / unique_tracks:.2f}")
    print(f"Interaction Matrix Density: {total_interactions / (unique_users * unique_tracks):.10f}")

# print_statistics(train_df)
# print_statistics(test_df)


# Load Dataset

1.   Direct download
2.   Github pull (default - train/test)



## 1. Direct download

In [ ]:
import pandas as pd
import os

flag_ds = '' #'lastfm1k_complete'
if flag_ds == 'lastfm_quick':
  # URL to the raw CSV file
  url = 'https://raw.githubusercontent.com/yasdel/RecSys_GPT/main/data/lastfm360k/mydata.csv'
  listening_data = pd.read_csv(url, index_col=0).reset_index(drop=True)
  listening_data.rename(columns={'user_id': 'userId', 'artist_id': 'artistId'}, inplace=True)
  # Check the first few rows of the DataFrame
  display(listening_data.head(20))

  url = 'https://raw.githubusercontent.com/yasdel/RecSys_GPT/main/data/lastfm360k/usersha1-profile.tsv'
  user_data = pd.read_table(url, header=None, names=['user', 'gender', 'age', 'country', 'signup'])

  # Cleaning user demographic data (Example steps; adjust as necessary)
  # Drop rows with missing user hash
  user_data = user_data.dropna(subset=['user'])
  # Assuming age is important, fill missing ages with the median or drop, depending on your needs
  user_data['age'] = user_data['age'].fillna(user_data['age'].median())
  most_frequent_gender = user_data['gender'].mode()[0]  # Find the most frequent gender
  user_data['gender'] = user_data['gender'].fillna(most_frequent_gender)  # Fill NaN values

  # Merge interaction data with user demographic data
  display(user_data.head(20))

elif flag_ds == 'lastfm_complete':

  !wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz
  #!wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz

  !ls
  !tar -xvzf lastfm-dataset-360K.tar.gz
  !ls

  import pandas as pd

  # Load interaction data
  raw_data = pd.read_table('lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv', header=None)
  raw_data.columns = ['user', 'artist_mb_id', 'artist', 'plays']
  # Rename columns for clarity
  # Drop rows with missing values
  listening_data = raw_data.dropna().copy()

  # Encode 'user' and 'artist' as categorical variables and create numeric ID columns
  listening_data['user'] = listening_data['user'].astype("category")
  listening_data['artist'] = listening_data['artist'].astype("category")
  listening_data['user_id'] = listening_data['user'].cat.codes
  listening_data['artist_id'] = listening_data['artist'].cat.codes
  listening_data.rename(columns={'user_id': 'userId', 'artist_id': 'artistId'}, inplace=True)

  # Load user demographic data
  user_data = pd.read_table('lastfm-dataset-360K/usersha1-profile.tsv', header=None, names=['user', 'gender', 'age', 'country', 'signup'])

  # Cleaning user demographic data (Example steps; adjust as necessary)
  # Drop rows with missing user hash
  user_data = user_data.dropna(subset=['user'])
  # Assuming age is important, fill missing ages with the median or drop, depending on your needs
  user_data['age'] = user_data['age'].fillna(user_data['age'].median())
  most_frequent_gender = user_data['gender'].mode()[0]  # Find the most frequent gender
  user_data['gender'] = user_data['gender'].fillna(most_frequent_gender)  # Fill NaN values

  # Merge interaction data with user demographic data
  listening_data['user'] = listening_data['user'].astype(str)
  user_data['user'] = user_data['user'].astype(str)

  # Merge the DataFrames
  full_data = pd.merge(listening_data, user_data, on='user', how='inner')
  full_data.head(10)

  # Creating a subset of full_data with just the user and userId columns
  user_ids = full_data[['user', 'userId']].drop_duplicates()

  # Merging user_data with the user_ids DataFrame to add the userId column
  # Using a left join to keep all rows in user_data
  user_data = pd.merge(user_data, user_ids, on='user', how='left')

elif flag_ds == 'lastfm1k_complete':
    # Download and extract data if not already done
    if not os.path.exists('lastfm-dataset-1K.tar.gz'):
        !wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz -O lastfm-dataset-1K.tar.gz
    if not os.path.exists('lastfm-dataset-1K'):
        !tar -xvzf lastfm-dataset-1K.tar.gz

    path_to_data = 'lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv'
    # Load data efficiently
    raw_data = pd.read_csv(
        path_to_data,
        delimiter='\t',
        header=0,
        names=['user', 'timestamp', 'artist_mb_id', 'artist', 'track_id', 'track_name'],
        on_bad_lines='skip',
        usecols=['user', 'timestamp', 'artist', 'track_id', 'track_name'],  # Exclude artist_mb_id to save memory
        dtype={'user': 'category', 'artist': 'category', 'track_id': 'category', 'track_name': 'category'},
        parse_dates=['timestamp']
    )

    # Save raw sampled data
    raw_data.sample(frac=1.0).to_csv('raw_data.csv', index=False)

    # Drop rows with any NaN values
    listening_data = raw_data.dropna()
    #listening_data.to_csv('listening_data.csv', index=False)

    # Convert 'user' and 'artist' to categorical and map them to codes
    listening_data['userId'] = listening_data['user'].cat.codes
    listening_data['artistId'] = listening_data['artist'].cat.codes
    #listening_data.to_csv('listening_data.csv', index=False)

    # Count unique IDs and other metrics
    unique_userIds = listening_data['userId'].nunique()
    unique_artistIds = listening_data['artistId'].nunique()
    unique_trackIds = listening_data['track_id'].nunique()

    # User profile data loading and cleaning
    user_profile_path = 'lastfm-dataset-1K/userid-profile.tsv'
    user_data = pd.read_csv(
        user_profile_path,
        delimiter='\t',
        header=0,
        names=['user', 'gender', 'age', 'country', 'signup'],
        usecols=['user', 'gender', 'age', 'country', 'signup'],
        dtype={'user': 'category', 'gender': 'category', 'country': 'category'},
        parse_dates=['signup']
    )

    user_data.dropna(subset=['user'], inplace=True)
    user_data['age'] = pd.to_numeric(user_data['age'], errors='coerce').fillna(user_data['age'].median())
    user_data['gender'].fillna(user_data['gender'].mode()[0], inplace=True)
    #user_data.to_csv('user_data.csv', index=False)

    # Merge listening and user data
    full_data = pd.merge(listening_data, user_data, on='user', how='inner')
    #full_data.to_csv('full_data.csv', index=False)

    # Save the merged data
    print(full_data.head(10))


In [ ]:
def apply_interaction_limits(data, min_interactions, max_interactions, user_col, item_col):
    # Count the number of interactions by user
    interaction_counts = data[user_col].value_counts()

    # Filter out users outside the interaction limits
    users_to_keep = interaction_counts[(interaction_counts >= min_interactions) & (interaction_counts <= max_interactions)].index
    return data[data[user_col].isin(users_to_keep)]

# Constants for interaction thresholds
MIN_INTERACTIONS_PER_USER = 50  # Minimum threshold for user activity
MAX_INTERACTIONS_PER_USER = 10000  # Maximum threshold to exclude superusers

# # Apply interaction limits
# limited_interaction_data = apply_interaction_limits(
#     full_data,
#     MIN_INTERACTIONS_PER_USER,
#     MAX_INTERACTIONS_PER_USER,
#     'user',
#     'track_name'
# )

# # limited_interaction_data
# print_statistics(full_data)
# print_statistics(limited_interaction_data)


In [ ]:
import pandas as pd
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

def temporal_train_test_split(data: pd.DataFrame, test_size=0.2) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the data for each user into training and test sets based on time (temporal split).

    Parameters:
        data (pd.DataFrame): The complete dataset containing at least 'user', 'timestamp' columns.
        test_size (float): The proportion of the dataset to include in the test split (0 to 1).

    Returns:
        train (pd.DataFrame): Training set.
        test (pd.DataFrame): Test set.
    """
    np.random.seed(0)  # Set a random seed for consistency in 'random' sampling
    data_sorted = data.sort_values(by=['user', 'timestamp'])

    def split_group(group):
        split_point = int((1 - test_size) * len(group))
        return group.iloc[:split_point], group.iloc[split_point:]

    train_list, test_list = zip(*data_sorted.groupby('user').apply(split_group))

    train = pd.concat(train_list).reset_index(drop=True)
    test = pd.concat(test_list).reset_index(drop=True)

    return train, test

# # Step 1: Randomly sample 200 unique users
# sampled_users = np.random.choice(limited_interaction_data['user'].unique(), 80, replace=False)

# # Step 2: Filter the data for these users
# sampled_data = limited_interaction_data[limited_interaction_data['user'].isin(sampled_users)]

# # Step 3: Apply the temporal train-test split function
# train_data, test_data = temporal_train_test_split(sampled_data)

# # Displaying the shapes of the outputs
# print("Train Data Shape:", train_data.shape)
# print("Test Data Shape:", test_data.shape)
# # limited_interaction_data
# print_statistics(train_data)
# print_statistics(test_data)


In [ ]:
# print_statistics(train_data)

# # Assuming train_data is your DataFrame
# num_rows = train_data.shape[0]
# chunk_size = num_rows // 4  # Integer division to get the size of each chunk

# # Split train_data into 4 parts and save
# for i in range(4):
#     start_index = i * chunk_size
#     # For the last chunk, make sure to include any remaining rows
#     end_index = (i + 1) * chunk_size if i < 3 else num_rows
#     chunk = train_data.iloc[start_index:end_index]
#     chunk.to_csv(f'train_data_lastFM1k_fullInteraction_80users_part_{i+1}.csv', index=False)

# test_data.to_csv('test_data_lastFM1k_fullInteraction_80users.csv')

## 2. Direct download (train/test)

In [ ]:
import requests
import pandas as pd
from IPython.display import display

def download_csv_from_github(file_url, local_path):
    """
    Download a file from GitHub using its raw URL and save it locally.

    Parameters:
    file_url (str): The raw URL to the file on GitHub.
    local_path (str): The local path where the file should be saved.
    """
    try:
        response = requests.get(file_url)
        response.raise_for_status()  # Will raise an HTTPError for bad requests (400 or 500)
        with open(local_path, 'wb') as f:
            f.write(response.content)
        print(f"Successfully downloaded {local_path}")
        return pd.read_csv(local_path)  # Return the dataframe
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")  # Specific HTTP error
    except Exception as err:
        print(f"An error occurred: {err}")  # General errors in the download process

# Base URL for raw files on GitHub
base_url = "https://raw.githubusercontent.com/yasdel/Benchmark_RecLLM_Fairness/main/data/"

files = [
    "test_data_lastFM1k_fullInteraction_80users.csv",
    "train_data_lastFM1k_fullInteraction_80users_part_1.csv",
    "train_data_lastFM1k_fullInteraction_80users_part_2.csv",
    "train_data_lastFM1k_fullInteraction_80users_part_3.csv",
    "train_data_lastFM1k_fullInteraction_80users_part_4.csv"
]


# Download and load all files into dataframes
dfs = [download_csv_from_github(base_url + file, file) for file in files]

# Concatenate all train data parts into one dataframe
train_df = pd.concat(dfs[1:], ignore_index=True)

# Test data dataframe
test_df = dfs[0]
test_df = test_df.drop(test_df.columns[0], axis=1)

# Print the first few rows of the concatenated train dataframe and test dataframe
print("Train DataFrame:")
display(train_df.head())

print("Test DataFrame:")
display(test_df.head())

Successfully downloaded test_data_lastFM1k_fullInteraction_80users.csv
Successfully downloaded train_data_lastFM1k_fullInteraction_80users_part_1.csv
Successfully downloaded train_data_lastFM1k_fullInteraction_80users_part_2.csv


In [ ]:
print_statistics(train_df)
print_statistics(test_df)


In [ ]:
train_df

In [ ]:
import pandas as pd

def prepare_data_with_item_id_dict(train_df, test_df):
    """
    Prepares track data for recommendation algorithms by ensuring unique track IDs,
    adds an 'itemId' column that is a numeric representation of the track_id directly,
    ensures 'itemId', 'userId', and 'artistId' are all integers, and creates a dictionary
    with itemId as keys mapping to {'track_name': ..., 'artist': ...}.

    Parameters:
    - train_df (pd.DataFrame): Training data set containing at least 'track_id', 'track_name', 'artist', 'userId', 'artistId'.
    - test_df (pd.DataFrame): Testing data set containing at least 'track_id', 'track_name', 'artist', 'userId', 'artistId'.

    Returns:
    - dict: A dictionary with itemId as keys and {'track_name': ..., 'artist': ...} as values.
    - pd.DataFrame: Updated training data frame with a new column 'itemId' and type-casted 'userId', 'artistId'.
    - pd.DataFrame: Updated testing data frame with a new column 'itemId' and type-casted 'userId', 'artistId'.
    - dict: A dictionary mapping alphanumeric 'track_id' to a unique numeric ID (itemId).
    """
    # Combine train and test data frames to ensure all track IDs are included
    all_tracks = pd.concat([train_df[['track_id', 'track_name', 'artist']], test_df[['track_id', 'track_name', 'artist']]])

    # Remove duplicates to ensure each track_id corresponds to a single track_name and artist
    unique_tracks = all_tracks.drop_duplicates(subset='track_id')

    # Create a numeric encoding for track IDs
    track_id_to_numeric = {tid: i for i, tid in enumerate(unique_tracks['track_id'].unique())}

    # Map the numeric IDs back to the original data frames and directly use 'itemId'
    train_df['itemId'] = train_df['track_id'].map(track_id_to_numeric).astype(int)
    test_df['itemId'] = test_df['track_id'].map(track_id_to_numeric).astype(int)

    # Ensure 'userId' and 'artistId' are integers
    train_df['userId'] = train_df['userId'].astype(int)
    train_df['artistId'] = train_df['artistId'].astype(int)
    test_df['userId'] = test_df['userId'].astype(int)
    test_df['artistId'] = test_df['artistId'].astype(int)

    # Create a dictionary with itemId as the key
    item_dict = unique_tracks.set_index('track_id').to_dict('index')
    item_dict = {track_id_to_numeric[tid]: val for tid, val in item_dict.items()}

    return item_dict, train_df, test_df, track_id_to_numeric


item_dict, train_df, test_df, id_map = prepare_data_with_item_id_dict(train_df, test_df)
# Define the mapping dictionary
gender_map = {
    'f': 'Female',
    'm': 'Male'
}

# Apply the mapping
train_df['gender'] = train_df['gender'].map(gender_map)
test_df['gender'] = test_df['gender'].map(gender_map)


In [ ]:
item_dict

In [ ]:
# import pandas as pd

# # Creating an expanded example DataFrame
# data = {
#     'timestamp': [
#         '2006-01-23 08:13:39+00:00', '2006-01-23 08:17:26+00:00', '2006-01-23 08:13:39+00:00',
#         '2006-01-23 08:28:59+00:00', '2006-01-23 08:13:39+00:00', '2006-01-23 08:28:59+00:00',
#         '2006-01-23 08:13:39+00:00', '2006-01-23 08:28:59+00:00', '2006-01-23 08:26:14+00:00'
#     ],
#     'artist': ['Chris Brown', 'Chris Brown', 'Chris Brown', 'Chris Brown', 'Bob James', 'Bob James', 'Boney James', 'Boney James', 'Shakedown'],
#     'track_id': [
#         '29fe3132-9025-4775-9f17-5432e53886c6', '29fe3132-9025-4775-9f17-5432e53886c6',
#         '29fe3132-9025-4775-9f17-5432e53886c6', '5459a342-c74f-4a6e-b978-89ef4dc30b81',
#         '212e46a0-de16-4c92-9b00-f4d9f506c5a6', '212e46a0-de16-4c92-9b00-f4d9f506c5a6',
#         'e873111e-e1a7-4885-8d6a-91126d476c25', 'e873111e-e1a7-4885-8d6a-91126d476c25',
#         '747f21ab-6480-4d48-b9a2-c4516ee20b07'
#     ],
#     'track_name': [
#         'Run It (Feat. Juelz Santana)', 'Run It (Feat. Juelz Santana)', 'Run It (Feat. Juelz Santana)',
#         "What'S My Name (Feat. Noah)", 'Take Me There', 'Take Me There', 'Sara Smile', 'Sara Smile', 'At Night (Afterlife Mix)'
#     ],
#     'userId': [6, 6, 6, 6, 7, 7, 8, 8, 8],
#     'artistId': [813, 813, 813, 813, 582, 582, 5328, 5328, 3610],
#     'gender': ['f', 'f', 'f', 'f', 'm', 'm', 'f', 'f', 'm'],
#     'age': [23.0, 23.0, 23.0, 23.0, 34.0, 34.0, 28.0, 28.0, 25.0],
#     'country': ['United States']*9,
#     'signup': ['2006-01-22']*9,
#     'itemId': [0, 0, 0, 3, 4, 4, 5, 5, 6]
# }
# df = pd.DataFrame(data)

# Group by 'userId', 'itemId', and 'artistId' and aggregate
aggregated_train_df = train_df.groupby(['userId', 'itemId', 'artistId']).agg({
    'timestamp': 'first',  # First timestamp
    'artist': 'first',     # First artist name
    'track_id': 'first',   # First track ID
    'track_name': 'first', # First track name
    'gender': 'first',     # First gender
    'age': 'first',        # First age
    'country': 'first',    # First country
    'signup': 'first',     # First signup
    'userId': 'size'       # Count of occurrences
}).rename(columns={'userId': 'count'}).reset_index()

# Group by 'userId', 'itemId', and 'artistId' and aggregate
aggregated_test_df = test_df.groupby(['userId', 'itemId', 'artistId']).agg({
    'timestamp': 'first',  # First timestamp
    'artist': 'first',     # First artist name
    'track_id': 'first',   # First track ID
    'track_name': 'first', # First track name
    'gender': 'first',     # First gender
    'age': 'first',        # First age
    'country': 'first',    # First country
    'signup': 'first',     # First signup
    'userId': 'size'       # Count of occurrences
}).rename(columns={'userId': 'count'}).reset_index()



# Display the resulting DataFrame
aggregated_train_df.head(10)


## Data Visualization

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


# Function to improve visualization for track and artist plays over time for a random user
def improved_color_visualization(df):
    # Select a random user
    random_user = np.random.choice(df['user'].unique())
    print(f"Visualizing track and artist plays over time for: {random_user}")

    # Filter data for the selected user
    user_data = df[df['user'] == random_user]
    print(user_data)

    # Convert timestamp to datetime and sort by it
    user_data['timestamp'] = pd.to_datetime(user_data['timestamp'])
    user_data = user_data.sort_values(by='timestamp')

    # Resample the data weekly for less granularity and count the number of tracks and unique artists
    user_data.set_index('timestamp', inplace=True)
    track_counts = user_data['track_name'].resample('W').count()  # Count of tracks each week
    artist_counts = user_data['artist'].resample('W').nunique()  # Count of unique artists each week

    # Combine the counts into a single DataFrame
    combined_counts = pd.concat([track_counts, artist_counts], axis=1)
    combined_counts.columns = ['Track Counts', 'Artist Counts']

    # Increase the size of the plot to make it less cluttered
    plt.figure(figsize=(15, 8))  # Larger figure size

    # Set the color for the bars with a grayscale and set alpha for density
    colors = ['gray', 'orange']

    # Plotting with specified colors and alpha for density
    combined_counts.plot(kind='bar', stacked=True, color=colors, alpha=0.75, figsize=(15, 8), width=0.6)
    plt.title(f'Track and Artist Plays Over Time for {random_user}', color='gray')
    plt.xlabel('Timestamp', color='gray')
    plt.ylabel('Counts', color='gray')
    plt.legend()
    plt.grid(True, color='lightgray', linestyle='--', linewidth=0.5)

    # Rotate the x-axis labels to vertical and set a less dense x-tick frequency
    plt.xticks(rotation=90, ha='center', color='gray')
    ax = plt.gca()
    temp = ax.xaxis.get_ticklabels()
    temp = list(set(temp) - set(temp[::4]))  # Hide every 4th label to reduce clutter
    for label in temp:
        label.set_visible(False)
    for label in ax.get_yticklabels():
        label.set_color('gray')

    plt.tight_layout()  # Adjust plot
    plt.show()

# Call the function to
# Call the function to visualize data with improvements
improved_color_visualization(train_df)
improved_color_visualization(test_df)

2. Github Pull (default)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Function to categorize age
def categorize_age(age, mean_age):
    if age <= mean_age:
        return f"Early Adult (≤{int(mean_age)} yrs)"
    else:
        return f"Mid Adult (>{int(mean_age)} yrs)"

# Calculating the mean age
mean_age = aggregated_train_df['age'].mean()
print(f"Mean Age: {mean_age:.2f} years")

# Applying the function to categorize ages
aggregated_train_df['age_group'] = aggregated_train_df['age'].apply(lambda x: categorize_age(x, mean_age))

# Plotting the distributions
plt.figure(figsize=(10, 6))
age_group_distribution = aggregated_train_df['age_group'].value_counts()
sorted_age_groups = age_group_distribution[['Early Adult (≤24 yrs)', 'Mid Adult (>24 yrs)']]
sorted_age_groups.plot(kind='bar')
plt.title('Distribution of Age Groups with Updated Boundaries')
plt.xlabel('Age Group')
plt.ylabel('Number of Users')
plt.xticks(rotation=0)
plt.show()

# Creating a pivot table and plotting it
pivot_data = aggregated_train_df.pivot_table(index='age_group', columns='gender', values='userId', aggfunc='size', fill_value=0)
pivot_data.plot(kind='bar', stacked=True, figsize=(12, 8))
plt.title('Intersectional Distribution of Gender and Age Groups - Stacked Bar Chart')
plt.xlabel('Age Group')
plt.ylabel('Number of Users')
plt.xticks(rotation=0)
plt.legend(title='Gender')
plt.show()


# **OpenAI Experiments**

## 1. Prompt generation

In [ ]:
!pip install openai

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

def generate_demographic_prompt(user_id, df_train, user_info, counterfactual):
    """
    Generates a demographic description based on user information, optionally with counterfactual alterations.
    """
    if user_info is None:
        return "The user has listenned to the following songs in the past, organized as (Song - Artist):\n"

    user_demo = df_train[df_train['userId'] == user_id].iloc[0]
    gender_desc = user_demo['gender']
    age_group_desc = user_demo['age_group']

    if counterfactual:
        if user_info in ['gender', 'intersectional']:
            gender_desc = "Female" if gender_desc.lower() == "male" else "Male"
        if user_info in ['age-group', 'intersectional']:
            age_group_desc = "Mid Adult (>24 yrs)" if age_group_desc == "Early Adult (≤24 yrs)" else "Early Adult (≤24 yrs)"

    descriptors = ''
    if user_info == 'gender':
        descriptors = f"The user is {gender_desc}."
    elif user_info == 'age-group':
        descriptors = f"The user is {age_group_desc}."
    elif user_info == 'intersectional':
        descriptors = f"The user is {gender_desc} and {age_group_desc}."

    return f"{descriptors} The user has listenned to the following songs in the past, organized as (Song - Artist):\n"

def passion_profile_generator(profile_prompt, user_tracks):
    """
    Formats the list of user's tracks into a structured output with "Song by Artist".

    Parameters:
    - profile_prompt (str): Introduction or context sentence provided to lead into the list of tracks.
    - user_tracks (list of dicts): List containing track details such as 'track_name' and 'artist'.

    Returns:
    - str: A formatted string combining the profile prompt with a list of songs followed by a closing statement reflecting the user's musical preferences.
    """
    track_artist_list = '\n'.join([f"- \"{track['track_name']}\" by {track['artist']}" for track in user_tracks])
    return f"{profile_prompt}\n{track_artist_list}\nThis selection reflects the user's musical preferences."

def generate_interaction_prompt(model, user_tracks, context_length=10, top_k=3):
    """
    Generates an interaction profile based on the specified model and the user's track history, focusing on the last 12 tracks for Few-shot context.
    """
    if model == 'zero-shot':
      sorted_tracks = sorted(user_tracks, key=lambda x: x.get('timestamp'), reverse=True)
      context_tracks = sorted_tracks[:context_length]
      track_listing = '\n'.join([f"{i+1}. \"{track['track_name']}\" by {track['artist']}" for i, track in enumerate(reversed(context_tracks))])
      if top_k==1:
        return f"Given the user's recent listening history listed below, \n{track_listing}\nwhich song would be a suitable next recommendation?"
      else:
        return f"Given the user's recent listening history listed below, \n{track_listing}\nwhat would be the top-{top_k} suitable next recommendations?"
    elif model == 'ICL-Few-shot-1':
      shot_count = int(model.split('-')[-1])
      if len(user_tracks) < context_length + 2:  # Adjust number of tracks needed
        return "Not enough tracks to generate a Few-shot profile."

      sorted_tracks = sorted(user_tracks, key=lambda x: x['timestamp'], reverse=True)
      context_tracks = sorted_tracks[1:context_length+1]  # Skip the most recent track to use as the example prediction
      example_track = sorted_tracks[0]  # The most recent track as the example

      context_list = '\n'.join([f"{i+1}. \"{track['track_name']}\" by {track['artist']}" for i, track in enumerate(reversed(context_tracks))])
      example_statement = f"Given the user has recently listened to the following songs in order:\n{context_list}\nYou should recommend:\n\"{example_track['track_name']}\" by {example_track['artist']}."
      if top_k==1:
        query_statement = f"What would be a suitable next recommendation after \"{example_track['track_name']}\" by {example_track['artist']}?"
      else:
        query_statement = f"What would be the top-{top_k} suitable next recommendations after \"{example_track['track_name']}\" by {example_track['artist']}?"

      return f"{example_statement}\n{query_statement}"
    elif model == 'ICL-Few-shot-2':
      shot_count = int(model.split('-')[-1])
      if len(user_tracks) < context_length + 3:  # Adjust number of tracks needed to include two examples and one query recommendation
          return "Not enough tracks to generate a Few-shot profile."

      sorted_tracks = sorted(user_tracks, key=lambda x: x['timestamp'], reverse=True)
      context_tracks = sorted_tracks[2:context_length+2]  # Context excludes the most recent two tracks used as examples
      example_track1 = sorted_tracks[1]  # The second most recent track as the first example
      example_track2 = sorted_tracks[0]  # The most recent track as the second example

      context_list = '\n'.join([f"{i+1}. \"{track['track_name']}\" by {track['artist']}" for i, track in enumerate(reversed(context_tracks))])
      example_statement1 = f"Given the user has recently listened to the following songs in order:\n{context_list}\nYou should recommend:\n\"{example_track1['track_name']}\" by {example_track1['artist']}."
      example_statement2 = f"After \"{example_track1['track_name']}\" by {example_track1['artist']}, you recommended:\n\"{example_track2['track_name']}\" by {example_track2['artist']}\n that the user liked."
      if top_k==1:
        query_statement = f"What would be a suitable next recommendation after \"{example_track2['track_name']}\" by {example_track2['artist']}?"
      else:
        query_statement = f"What would be the top-{top_k} suitable next recommendations after \"{example_track2['track_name']}\" by {example_track2['artist']}?"

      return f"{example_statement1}\n{example_statement2}\n{query_statement}"

def userProfile_Prompt_Gen(user_id, df_train, num_tracks=10, sampling_strategy='random', user_info=None,
                           counterfactual=False, interaction_model='zero-shot', context_length=10, top_k=3):
    """
    Generates a profile of a user's listened tracks, incorporating passion and interaction profiles,
    for the purpose of evaluating fairness (but also accuracy) in recommendation systems using pre-trained
    language models (pLLMs). This function first samples tracks based on the specified strategy (random, frequent, or recent-frequent),
    constructs a profile composed of the following three elements

    The combined profile is constructed as:
    combined_prompt = demographic_prompt + passion_profile + interaction_profile

    where
      . 'demographic_prompt' is a simple string reflecting user gender and age-group e.g., "the user is Male" or "the User is Young", (Fairness Specific)
      . 'passion_profile' reflecting user's general historical preference (the profile could be long, therefore 3 sampling startegies are placed)
      . 'interaction_profile' reflecting most recent interactions  (consist of Zero-shot, and ICL-1)

    Parameters:
    - user_id (int): The user's unique identifier.
    - df_train (DataFrame): The dataset containing all user interactions.
    - num_tracks (int): The number of tracks to sample for the passion profile.
    - sampling_strategy (str): The strategy for sampling tracks for the passion profile ('random', 'frequent', 'recent-frequent').
    - user_info (str, optional): Specifies if additional user demographic info (gender or age-group) should be included.
    - counterfactual (bool): If True, demographic details are altered (e.g., switching gender), used for hypothetical scenarios to assess fairness.
    - interaction_model (str): The type of interaction model to use ('zero-shot', 'ICL-Few-shot-#').
    - context_length (int): The number of tracks to consider for generating interaction profiles in few-shot settings.

    Returns:
    - str: A string combining the user's passion profile (musical preferences) and interaction profile (model interactions), used to evaluate the LLM's performance and fairness.

    Sampling strategies for the passion profile:
    - 'random': Tracks are sampled randomly.
    - 'frequent': Tracks are sampled based on play count, prioritizing frequently played tracks.
    - 'recent-frequent': Combines recency and frequency where:
      score = 0.7 * normalized_recency + 0.3 * normalized_count
      Here, normalized_recency = 1 - ((recency_score - min(recency_score)) / (max(recency_score) - min(recency_score)))
      and normalized_count = (count - min(count)) / (max(count) - min(count))
      This formula helps in selecting tracks that are both recent and frequently listened to.
    """
    #introduction = "You are going to see a user's musical profile. This profile is divided into two main sections: the Passion Profile, which showcases historical musical preferences, and the Recent Interactions, showing recently listened tracks.\n\n"

    np.random.seed(0)  # Consistent sampling
    user_listened_items = df_train[df_train['userId'] == user_id].copy()
    available_tracks = len(user_listened_items)
    num_to_sample = min(num_tracks, available_tracks)

    if sampling_strategy == 'random':
        sampled_items = user_listened_items.sample(n=num_to_sample, replace=False)
    elif sampling_strategy == 'frequent':
        sampled_items = user_listened_items.sort_values(by='count', ascending=False).head(num_to_sample)
    elif sampling_strategy == 'recent-frequent':
        current_timestamp = pd.to_datetime("now", utc=True)
        user_listened_items['timestamp'] = pd.to_datetime(user_listened_items['timestamp'], utc=True)
        # current_timestamp = pd.to_datetime("now")
        # user_listened_items['timestamp'] = pd.to_datetime(user_listened_items['timestamp'])
        user_listened_items['recency_score'] = (current_timestamp - user_listened_items['timestamp']).dt.total_seconds() / 86400
        user_listened_items['normalized_count'] = (user_listened_items['count'] - user_listened_items['count'].min()) / (user_listened_items['count'].max() - user_listened_items['count'].min())
        user_listened_items['normalized_recency'] = 1 - ((user_listened_items['recency_score'] - user_listened_items['recency_score'].min()) / (user_listened_items['recency_score'].max() - user_listened_items['recency_score'].min()))
        recency_weight = 0.65
        count_weight = 0.35
        user_listened_items['combined_score'] = (recency_weight * user_listened_items['normalized_recency']) + (count_weight * user_listened_items['normalized_count'])
        sampled_items = user_listened_items.nlargest(num_to_sample, 'combined_score')

    sampled_tracks = [{'artist': row['artist'], 'track_name': row['track_name'], 'timestamp': row['timestamp']} for _, row in sampled_items.iterrows()]
    all_tracks = [{'artist': row['artist'], 'track_name': row['track_name'], 'timestamp': row['timestamp']} for _, row in user_listened_items.iterrows()]

    # Generating the passion profile
    profile_prompt = generate_demographic_prompt(user_id, df_train, user_info, counterfactual)
    passion_profile_title = "Passion profile:"
    passion_profile = passion_profile_generator(profile_prompt, sampled_tracks)
    interaction_profile_title = "Interaction profile:"
    interaction_profile = generate_interaction_prompt(interaction_model, all_tracks, context_length, top_k)
    #combined_profile = f"{introduction}\n{passion_profile_title}\n{passion_profile}\n\n{interaction_profile_title}{interaction_profile}"
    combined_profile = f"{passion_profile}\n{interaction_profile}"
    return combined_profile



###############################################################################
############## Runs OpenAI Experiments (from Train or Toy Exp.)  ##############
###############################################################################
# Create mock data for testing
data = {
    'userId': [7]*15,
    'artist': ['Artist1', 'Artist2', 'Artist3', 'Artist4', 'Artist5',
               'Artist6', 'Artist7', 'Artist8', 'Artist9', 'Artist10',
               'Artist11', 'Artist12', 'Artist13', 'Artist14', 'Artist15'],
    'track_name': ['Track1', 'Track2', 'Track3', 'Track4', 'Track5',
                   'Track6', 'Track7', 'Track8', 'Track9', 'Track10',
                   'Track11', 'Track12', 'Track13', 'Track14', 'Track15'],
    'timestamp': [datetime.now() - timedelta(days=i) for i in range(15)],
    'count': [5, 1, 3, 7, 2, 2, 1, 4, 6, 1, 1, 2, 3, 4, 5],
    'gender': ['Male']*15,
    'age_group': ['Early Adult (≤24 yrs)']*15
}
df_train = pd.DataFrame(data)

df_train = aggregated_train_df.copy()
print(df_train)

df = df_train.sample(frac=1).reset_index(drop=True)
user_id = df.loc[0, 'userId']

# Select all rows corresponding to this user
first_user_data = df[df['userId'] == user_id]
# Display the data for the first user
print(first_user_data)

# Define user ID and number of tracks to sample
num_tracks = 10
context_length = 5
top_k = 3
# Define parameters for testing
sampling_strategy_list = ['random', 'frequent', 'recent-frequent']
user_infos_list = [None, 'gender', 'age-group', 'intersectional']
counterfactual_list = [False, True]
interaction_model_options = ['zero-shot', 'ICL-Few-shot-1', 'ICL-Few-shot-2']


if run_openAI_experiments == 1:
  # Call the function with test parameters
  for counterfactual in counterfactual_list:
    for sampling_strategy in sampling_strategy_list:
      for user_infos in user_infos_list:
        for interaction_model in interaction_model_options:
          profile_output = userProfile_Prompt_Gen(
              user_id, df_train, num_tracks, sampling_strategy, user_infos, counterfactual, interaction_model, context_length=context_length, top_k=top_k
          )
          print(profile_output)


In [ ]:
run_openAI_experiments

# Run Prompt Generation for ALL Users in Train Dataset  

In [ ]:
# Assuming 'train_new_df' is already defined as your initial DataFrame
df_train = aggregated_train_df.copy()

# Define parameters for testing
sampling_strategy_list = ['random', 'frequent','recent-frequent']
user_infos_list = [None, 'gender', 'age-group', 'intersectional']
counterfactual_list = [False, True]
interaction_model_options = ['zero-shot', 'ICL-Few-shot-1', 'ICL-Few-shot-2']
num_tracks = 10
context_length = 5
top_k = 10
# Create a new DataFrame to store results grouped by user ID
prompt_df = pd.DataFrame()


if load_trained_data == 0:
  # Iterate over each unique user to generate prompts
  for user_id in df_train['userId'].unique():
      # Extract user specific data
      user_data = df_train[df_train['userId'] == user_id]
      # Basic user info
      gender = user_data['gender'].iloc[0]  # Assuming gender is the same for all rows of the same user
      age_group = user_data['age_group'].iloc[0]  # Assuming age_group is consistent
      itemIds = list(user_data['itemId'].unique())  # List of unique artist IDs for the user
      artistIds = list(user_data['artistId'].unique())  # List of unique artist IDs for the user

      # Initialize the row to append to prompt_df
      row = pd.DataFrame({
          'userId': [user_id],
          'gender': [gender],
          'age_group': [age_group],
          'itemIds': [itemIds],
          'artistIds': [artistIds]
      })

      # Generate prompts based on various scenarios
      for counterfactual in counterfactual_list:
          for sampling_strategy in sampling_strategy_list:
              for user_info in user_infos_list:
                  for interaction_model in interaction_model_options:
                      column_name = f"prompt_counterfact-{counterfactual}_sample-{sampling_strategy}_userDemo-{user_info}_interact-{interaction_model}".replace('None', 'no_info')
                      profile_output = userProfile_Prompt_Gen(
              user_id, df_train, num_tracks, sampling_strategy, user_info, counterfactual, interaction_model, context_length=context_length, top_k=top_k
                      )
                      print(profile_output)
                      row[column_name] = [profile_output]  # Add the prompt to the row

      # Append the generated row to the results DataFrame
      prompt_df = pd.concat([prompt_df, row], ignore_index=True)

  # Display the first few rows of the results DataFrame
  print(prompt_df.head())

  # Optionally, save the updated dataframe to a new CSV file
  prompt_df.to_csv(f"train_prompt_df_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}_top_{top_k}.csv", index=False)
else:
  # Base URL for raw files on GitHub
  base_url = "https://raw.githubusercontent.com/yasdel/Benchmark_RecLLM_Fairness/main/data/"
  file = f"train_prompt_df_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}_top_{top_k}.csv"
  #file = f"train_prompt_df_lastFM1k_fullInteraction_80users_nTrack10_lContext5.csv"
  prompt_df = download_csv_from_github(base_url + file, file)
  prompt_df.head(10)


In [ ]:
import pandas as pd
import numpy as np

# Select a random user's index. This will choose one user at random using their integer location.
random_index = np.random.randint(0, len(prompt_df))

# Select the data for this random user
random_user_data = prompt_df.iloc[random_index]

# Print the data for this random user
print("Data for User at Random Index {}:".format(random_index))
for column, value in random_user_data.items():
    print(f"{column}: {value}")
    print("-------------------------------------------")


In [ ]:
prompt_df

In [ ]:
from openai import OpenAI
import pandas as pd
import time  # Add the time module for latency measurement

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=api_key)

openai_api_rate = 0.02  # Example rate: $0.02 per 1k tokens (replace with the actual rate you are using)

def get_recommendation(prompt):
    system_message = {"role": "system", "content": "Given a user, act like a Recommender System."}
    user_message = {"role": "user", "content": prompt}
    messages = [system_message, user_message]

    start_time = time.time()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=1500,
        temperature=0
    )

    end_time = time.time()
    latency = end_time - start_time
    api_cost = (response['usage']['total_tokens'] / 1000) * openai_api_rate  # Calculate cost based on tokens used

    # Correctly accessing properties of the response object
    recommendation = response.choices[0].message.content.strip()  # Use dot notation to access 'content'
    return recommendation, latency, api_cost

sample_user_df = prompt_df.copy()

if run_openAI_experiments == 1:
    # Initialize an empty DataFrame for recommendations and costs
    recommendations_df = pd.DataFrame(index=sample_user_df.index)
    costs_df = pd.DataFrame(index=sample_user_df.index, columns=['latency', 'api_cost'])

    # Iterate over the columns for which you need to generate recommendations (4th column onwards)
    for column in sample_user_df.columns[5:]:
        recommendations = []  # This will hold the column data until ready to assign to DataFrame
        latencies = []
        api_costs = []

        # Iterate over each row in the DataFrame
        for index, row in sample_user_df.iterrows():
            prompt = row[column]
            try:
                recommendation, latency, api_cost = get_recommendation(prompt)
            except Exception as e:
                recommendation = f"An error occurred: {e}"
                latency, api_cost = 0, 0

            recommendations.append(recommendation)
            latencies.append(latency)
            api_costs.append(api_cost)

            # Print a message every 40 users
            if (index + 1) % 40 == 0:
                print(f"Processed {index + 1} users for column {column}")

        # After collecting all recommendations for a column, add them to the recommendations DataFrame
        recommendations_df[f'recommendation_for_{column}'] = recommendations
        costs_df.loc[:, 'latency'] = latencies
        costs_df.loc[:, 'api_cost'] = api_costs

    # Concatenate the original DataFrame with the new recommendations and costs DataFrames
    RecLLM_df = pd.concat([sample_user_df, recommendations_df, costs_df], axis=1)

    # Display the DataFrame with prompts, recommendations, and costs
    print(RecLLM_df)
    # Optionally, save the result to a CSV file
    RecLLM_df.to_csv(f"RecLLM_df_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}_top{top_k}.csv", index=False)

else:
    # Base URL for raw files on GitHub
    base_url = "https://raw.githubusercontent.com/yasdel/Benchmark_RecLLM_Fairness/main/results/"
    file = f"RecLLM_df_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}_top{top_k}.csv"
    #file = "RecLLM_df_lastFM1k_fullInteraction_80users_nTrack10_lContext5.csv"
    RecLLM_df = download_csv_from_github(base_url + file, file)
    RecLLM_df.head(10)


In [ ]:
# from openai import OpenAI
# import pandas as pd

# # Initialize the OpenAI client with your API key
# client = OpenAI(api_key=api_key)

# openai_api_rate = 0.02  # Example rate: $0.02 per 1k tokens (replace with the actual rate you are using)

# def get_recommendation(prompt):
#     system_message = {"role": "system", "content": "Given a user, act like a Recommender System."}
#     user_message = {"role": "user", "content": prompt}
#     messages = [system_message, user_message]

#     start_time = time.time()
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=messages,
#         max_tokens=1500,
#         temperature=0
#     )

#     end_time = time.time()
#     latency = end_time - start_time
#     api_cost = (response['usage']['total_tokens'] / 1000) * openai_api_rate  # Calculate cost based on tokens used

#     # Correctly accessing properties of the response object
#     recommendation = response.choices[0].message.content.strip()  # Use dot notation to access 'content'
#     return recommendation, latency, api_cost

# sample_user_df = prompt_df.copy()

# if run_openAI_experiments == 1:
#   # Initialize an empty DataFrame for recommendations
#   recommendations_df = pd.DataFrame(index=sample_user_df.index)

#   # Iterate over the columns for which you need to generate recommendations (4th column onwards)
#   for column in sample_user_df.columns[5:]:
#       recommendations = []  # This will hold the column data until ready to assign to DataFrame

#       # Iterate over each row in the DataFrame
#       for index, row in sample_user_df.iterrows():
#           prompt = row[column]
#           try:
#               recommendation = get_recommendation(prompt)
#           except Exception as e:
#               recommendation = f"An error occurred: {e}"

#           recommendations.append(recommendation)

#           # Print a message every 10 users
#           if (index + 1) % 40 == 0:
#               print(f"Processed {index + 1} users for column {column}")

#       # After collecting all recommendations for a column, add them to the recommendations DataFrame
#       recommendations_df[f'recommendation_for_{column}'] = recommendations

#   # Concatenate the original DataFrame with the new recommendations DataFrame
#   RecLLM_df = pd.concat([sample_user_df, recommendations_df], axis=1)

#   # Display the DataFrame with prompts and recommendations
#   print(RecLLM_df)
#   # Optionally, save the result to a CSV file
#   RecLLM_df.to_csv(f"RecLLM_df_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}_top{top_k}.csv", index=False)

# else:
#   # Base URL for raw files on GitHub
#   base_url = "https://raw.githubusercontent.com/yasdel/Benchmark_RecLLM_Fairness/main/results/"
#   file = f"RecLLM_df_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}_top{top_k}.csv"
#   #file = "RecLLM_df_lastFM1k_fullInteraction_80users_nTrack10_lContext5.csv"
#   RecLLM_df = download_csv_from_github(base_url + file, file)
#   RecLLM_df.head(10)

Processed 40 users for column prompt_counterfact-False_sample-random_userDemo-no_info_interact-zero-shot
Processed 80 users for column prompt_counterfact-False_sample-random_userDemo-no_info_interact-zero-shot
Processed 40 users for column prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-1
Processed 80 users for column prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-1
Processed 40 users for column prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-2
Processed 80 users for column prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-2
Processed 40 users for column prompt_counterfact-False_sample-random_userDemo-gender_interact-zero-shot
Processed 80 users for column prompt_counterfact-False_sample-random_userDemo-gender_interact-zero-shot
Processed 40 users for column prompt_counterfact-False_sample-random_userDemo-gender_interact-ICL-Few-shot-1
Processed 80 users for column pro

In [ ]:
import pandas as pd
import numpy as np

# Select a random user's index. This will choose one user at random using their integer location.
random_index = np.random.randint(0, len(RecLLM_df))

# Select the data for this random user
random_user_data = RecLLM_df.iloc[random_index]

# Print the data for this random user
print("Data for User at Random Index {}:".format(random_index))
for column, value in random_user_data.items():
    print(f"{column}: {value}")
    print("-------------------------------------------")


In [ ]:
RecLLM_df

#Evaluation

In [ ]:
api_key = "" #replace it with you own api key

In [ ]:
RecLLM_df

In [ ]:
load_trained_data=0

In [ ]:
import openai
import pandas as pd
#"content": "Please provide the recommended song and its artist from the following text in the format: \"Song Name\" by Artist."


def LLM_parsing(text, api_key):
    client = openai.OpenAI(api_key=api_key)
    try:
        system_message = {
            "role": "system",
            "content": "Please list the recommended song(s) and their artist(s) from the following text, each recommendation in the format: \"Song Name\" by Artist. If there are multiple recommendations, list each separated by a newline."
            #"content": "Please provide the recommended song(s) and its artist(s) from the following text, each recommendation in the format: \"Song Name\" by Artist."
        }
        user_message = {
            "role": "user",
            "content": f"{text}"
        }
        messages = [system_message, user_message]

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=150,
            temperature=0
        )
        recommendation = response.choices[0].message.content.strip()
        if "by" in recommendation:  # Simple check to see if the format seems correct
            return recommendation
        else:
            return None  # Format not as expected
    except Exception as e:
        print(f"Error processing text: {e}")
        return None  # Return None in case of error

def apply_LLM_to_all_users(df, api_key):
    # Identify all columns that start with 'recommendation'
    recommendation_columns = [col for col in df.columns if col.startswith('recommendation')]

    # Create an empty DataFrame to collect all new rows
    new_rows_list = []

    # Process each user in the DataFrame
    for index, row in df.iterrows():
        user_id = row['userId']
        new_row = row[['userId', 'gender', 'age_group', 'itemIds','artistIds']].copy()  # Start with basic user info

        # Apply LLM_parsing to each recommendation column for this user
        for col in recommendation_columns:
            try:
                original_text = row[col]  # Extract the text from the dataframe
                parsed_recommendation = LLM_parsing(original_text, api_key)  # Parse using LLM_parsing
                new_row[col] = parsed_recommendation  # Update the parsed recommendation column in the new row
            except Exception as e:
                print(f"Failed to process column {col} for user {user_id}: {e}")
                new_row[col] = None  # Set to None if any failure occurs

        new_rows_list.append(new_row)  # Append the newly created row to the list

    # Create a new DataFrame from the list of new rows
    new_df = pd.DataFrame(new_rows_list)
    return new_df

if load_trained_data == 0:
  # Assuming df is your DataFrame named finalPrompt_df
  RecLLM_parsed_df = apply_LLM_to_all_users(RecLLM_df, api_key)
  #RecLLM_parsed_df.to_csv(f"RecLLM_parsed_df_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}.csv", index=False)
  RecLLM_parsed_df.to_csv(f"RecLLM_parsed_df_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}_top{top_k}.csv", index=False)
else:
  #file = f"RecLLM_parsed_df_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}.csv"
  file = f"RecLLM_parsed_df_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}_top{top_k}.csv"
  RecLLM_parsed_df = download_csv_from_github(base_url + file, file)


ModuleNotFoundError: No module named 'openai'

In [ ]:
from google.colab import files

# Trigger the download
files.download('/content/'+ file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np

# Select a random user's index. This will choose one user at random using their integer location.
random_index = np.random.randint(0, len(RecLLM_parsed_df))

# Select the data for this random user
random_user_data = RecLLM_parsed_df.iloc[random_index]

# Print the data for this random user
print("Data for User at Random Index {}:".format(random_index))
for column, value in random_user_data.items():
    print(f"{column}: {value}")
    print("-------------------------------------------")


Data for User at Random Index 72:
userId: 904
-------------------------------------------
gender: Male
-------------------------------------------
age_group: Early Adult (≤24 yrs)
-------------------------------------------
itemIds: [180, 503, 512, 513, 514, 515, 516, 517, 520, 531, 535, 549, 578, 584, 663, 668, 688, 717, 723, 759, 763, 788, 793, 804, 806, 809, 812, 840, 841, 851, 876, 956, 981, 983, 988, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1038, 1073, 1132, 1133, 1134, 1135, 1140, 1142, 1206, 1210, 1233, 1288, 1312, 1328, 1366, 1367, 1368, 1369, 1398, 1409, 1410, 1453, 1579, 1580, 1581, 1582, 1583, 1584, 1591, 1598, 1601, 1630, 1631, 1632, 1633, 2140, 2156, 2488, 2543, 2622, 2908, 2909, 2910, 2911, 2912, 2913, 2914, 2936, 3108, 3346, 3519, 3574, 3603, 3605, 3606, 3607, 3608, 3609, 3610, 3611, 3614, 3615, 3616, 3617, 3618, 3619, 3620, 3621, 3622, 3623, 3624, 3626, 3628, 3629, 3630, 3631, 3695, 3704, 3717, 3731, 3939, 3982, 3985, 3986, 3987, 3988, 3989, 3990, 3991, 3992, 3993, 399

In [ ]:
RecLLM_parsed_df

,userId,gender,age_group,itemIds,artistIds,recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-zero-shot,recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-zero-shot,recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-ICL-Few-shot-1,...,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-no_info_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-gender_interact-zero-shot,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-gender_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-gender_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-age-group_interact-zero-shot,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-age-group_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-age-group_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-intersectional_interact-zero-shot,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-intersectional_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-intersectional_interact-ICL-Few-shot-2
0,6,Female,Early Adult (≤24 yrs),"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[813, 8857, 6855, 7218, 2643, 7105, 7728, 4990...","""No Scrubs"" by TLC\n""Waterfalls"" by TLC\n""Say ...","""Let's Get It On"" by Marvin Gaye \n""No Scrubs...","""Locked Up"" by Akon\n""Lonely"" by Akon\n""Smack ...","""No Scrubs"" by TLC\n""Waterfalls"" by TLC\n""Say ...","""Let's Get It On"" by Marvin Gaye \n""No Scrubs...",...,"""Don't Matter"" by Akon\n""Beautiful"" by Akon ft...","""Certified"" by Lloyd\n""My Love"" by Joe\n""One F...","""My Love"" by Joe \n""One For Me"" by Lloyd \n""...","""Smack That"" by Akon ft. Eminem\n""Don't Matter...","""Certified"" by Lloyd\n""My Love"" by Joe\n""One F...","""My Love"" by Joe \n""One For Me"" by Lloyd \n""...","""Smack That"" by Akon ft. Eminem\n""Don't Matter...","""Certified"" by Lloyd\n""My Love"" by Joe\n""One F...","""My Love"" by Joe \n""One For Me"" by Lloyd \n""...","""Smack That"" by Akon ft. Eminem\n""Don't Matter..."
1,46,Female,Early Adult (≤24 yrs),"[13, 74, 77, 155, 195, 203, 204, 217, 341, 345...","[4990, 2187, 6752, 4270, 2247, 2599, 3399, 502...","""The Trooper"" by Iron Maiden\n""Symphony of Des...","""The Trooper"" by Iron Maiden\n""Master of Puppe...","""The Trooper"" by Iron Maiden\n""Master of Puppe...","""Master of Puppets"" by Metallica \n""The Numbe...","""Master of Puppets"" by Metallica \n""The Numbe...",...,"""The Trooper"" by Iron Maiden\n""Holy Diver"" by ...","""All My Loving"" by Helloween\n""My Life For One...","""Get It Up"" by Helloween\n""My Life For One Mor...","""The Trooper"" by Iron Maiden \n""Holy Diver"" b...","""Shade In The Shadow"" by Helloween","""Get It Up"" by Helloween \n""Shade In The Shad...","""The Trooper"" by Iron Maiden \n""Holy Diver"" b...","""All My Loving"" by Helloween\n""My Life For One...","""Get It Up"" by Helloween \n""Shade In The Shad...","""The Trooper"" by Iron Maiden \n""Holy Diver"" b..."
2,48,Male,Early Adult (≤24 yrs),"[357, 1706, 1707, 1708, 1709, 1710, 1711, 1712...","[5237, 7143, 23731, 9565, 22461, 224, 18603, 3...","""Vordhosbn"" by Aphex Twin\n""Roygbiv"" by Boards...","""VLetrmx21"" by Autechre\n""Second Bad Vilbel"" b...","""VLetrmx21"" by Autechre\n""Sixtyten"" by Boards ...","""VLetrmx21"" by Autechre\n""Sixtyten"" by Boards ...","""Second Bad Vilbel"" by Autechre\n""Sixtyten"" by...",...,"""Spring And October"" by Weathertunes \n""6 

In [ ]:
item_dict

{0: {'track_name': 'Run It (Feat. Juelz Santana)', 'artist': 'Chris Brown'},
 1: {'track_name': 'Yo (Excuse Me Miss)', 'artist': 'Chris Brown'},
 2: {'track_name': 'Is This Love?', 'artist': 'Chris Brown'},
 3: {'track_name': "What'S My Name (Feat. Noah)", 'artist': 'Chris Brown'},
 4: {'track_name': "Ya Man Ain'T Me", 'artist': 'Chris Brown'},
 5: {'track_name': 'Gimme That', 'artist': 'Chris Brown'},
 6: {'track_name': 'Winner', 'artist': 'Chris Brown'},
 7: {'track_name': 'Sweet Lady', 'artist': 'Tyrese'},
 8: {'track_name': 'Love Calls', 'artist': 'Kem'},
 9: {'track_name': 'Let Me Love You (Album)', 'artist': 'Mario'},
 10: {'track_name': 'Be Without You', 'artist': 'Mary J. Blige'},
 11: {'track_name': 'Watching You', 'artist': 'Loose Ends'},
 12: {'track_name': 'Je Sais', 'artist': "Perle Lama & Princess' Lover"},
 13: {'track_name': 'Belly Dancer', 'artist': 'Akon'},
 14: {'track_name': 'Motema', 'artist': 'Kaysha'},
 15: {'track_name': 'One Love', 'artist': 'Kaysha'},
 16: {'t

In [ ]:
RecLLM_parsed_df

,userId,gender,age_group,itemIds,artistIds,recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-zero-shot,recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-zero-shot,recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-ICL-Few-shot-1,...,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-no_info_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-gender_interact-zero-shot,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-gender_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-gender_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-age-group_interact-zero-shot,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-age-group_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-age-group_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-intersectional_interact-zero-shot,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-intersectional_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-intersectional_interact-ICL-Few-shot-2
0,6,Female,Early Adult (≤24 yrs),"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[813, 8857, 6855, 7218, 2643, 7105, 7728, 4990...","""Un-break My Heart"" by Toni Braxton\n""Ex-Facto...","""Locked Up"" by Akon\n""So Lonely"" by Akon ft. E...","""Locked Up"" by Akon\n""Smack That"" by Akon ft. ...","""No Scrubs"" by TLC\n""Say My Name"" by Destiny's...","""Beautiful"" by Akon ft. Colby O'Donis & Kardin...",...,"""Smack That"" by Akon ft. Eminem\n""Don't Matter...","""One For Me"" by Lloyd \n""You"" by Lloyd ft. Li...","""Locked Up"" by Akon\n""I Wanna Love You"" by Ako...","""Don't Matter"" by Akon \n""Beautiful"" by Akon ...","""Certified"" by Lloyd \n""I'm So Fly"" by Lloyd ...","""Locked Up"" by Akon\n""Don't Matter"" by Akon\n""...","""Smack That"" by Akon ft. Eminem\n""Don't Matter...","""Certified"" by Lloyd \n""Bed"" by J. Holiday \...","""Locked Up"" by Akon\n""Don't Matter"" by Akon\n""...","""Don't Matter"" by Akon \n""Beautiful"" by Akon ..."
1,46,Female,Early Adult (≤24 yrs),"[13, 74, 77, 155, 195, 203, 204, 217, 341, 345...","[4990, 2187, 6752, 4270, 2247, 2599, 3399, 502...","""Master of Puppets"" by Metallica\n""Ov Fire and...","""Master of Puppets"" by Metallica\n""The Trooper...","""The Trooper"" by Iron Maiden\n""Breaking The La...","""Master of Puppets"" by Metallica\n""Ov Fire and...","- ""Ace of Spades"" by Motörhead\n- ""Holy Diver""...",...,"""Holy Diver"" by Dio\n""Ace of Spades"" by Motörh...","""Keeper of the Seven Keys"" by Helloween \n""Ea...","""Roundabout"" by Yes\n""Stairway to Heaven"" by L...","""Holy Diver"" by Dio\n""Ace of Spades"" by Motörh...","""I Want Out"" by Helloween\n""Halloween"" by Hell...","""Roundabout"" by Yes\n""Stargazer"" by Rainbow\n""...","""The Trooper"" by Iron Maiden\n""Ace of Spades"" ...","""I Want Out"" by Helloween\n""Halloween"" by Hell...","""Get It Up"" by Helloween\n""My Life For One Mor...","""The Trooper"" by Iron Maiden\n""Ace of Spades"" ..."
2,48,Male,Early Adult (≤24 yrs),"[357, 1706, 1707, 1708, 1709, 1710, 1711, 1712...","[5237, 7143, 23731, 9565, 22461, 224, 18603, 3...","""Roygbiv"" by Boards Of Canada\n""Dayvan Cowboy""...","""Second Bad Vilbel"" by Autechre \n""Gantz Graf...","""VLetrmx21"" by Autechre\n""Second Bad Vilbel"" b...","""VLetrmx21"" by Autechre\n""Sixtyten"" by Boards ...","""Second Bad Vilbel"" by Autechre\n""Clipper"" by ...",...,"""Second Bad Vilbel"" by

In [ ]:
import pandas as pd
import difflib

def find_closest_match(track, artist, item_dict, threshold=0.7):
    best_score = 0
    best_id = -1
    for item_id, details in item_dict.items():
        track_score = difflib.SequenceMatcher(None, track, details['track_name']).ratio()
        artist_score = difflib.SequenceMatcher(None, artist, details['artist']).ratio()
        score = (track_score + artist_score) / 2
        if score > best_score:
            best_score = score
            best_id = item_id
            if best_score >= threshold:
                break
    return best_id if best_score >= threshold else -1

def process_recommendations(df, item_dict, threshold=0.7):
    rec_columns = [col for col in df.columns if col.startswith("recommendation")]
    print(f"Processing {len(rec_columns)} recommendation columns.")

    # Initialize the columns to store the results
    for col in rec_columns:
        df[col + '_itemIds'] = [[] for _ in range(len(df))]

    for index, row in df.iterrows():
        for col in rec_columns:
            recommendations = row[col]
            song_ids = []
            if pd.notna(recommendations):
                songs = recommendations.split("\n")
                for song in songs:
                    parts = song.split(" by ")
                    if len(parts) == 2:
                        track, artist = parts
                        item_id = find_closest_match(track.strip(), artist.strip(), item_dict, threshold)
                        if item_id != -1:
                            song_ids.append(item_id)
            # Update the DataFrame with found item IDs
            df.at[index, col + '_itemIds'] = song_ids
            if index%5==0:  # Print example matches for the first few rows
                print(f"Row {index} - Column {col}: Found {len(song_ids)} valid songs.")

    return df

df = RecLLM_parsed_df.copy()
# Example usage assuming 'df' and 'item_dict' are defined
final_df = process_recommendations(df, item_dict, threshold=0.5)
file = f"final_df_foundItemIds_lastFM1k_fullInteraction_80users_nTrack{num_tracks}_lContext{context_length}_top{top_k}.csv"
final_df.to_csv(file, index=False)


Processing 72 recommendation columns.
Row 0 - Column recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-zero-shot: Found 2 valid songs.
Row 0 - Column recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-1: Found 10 valid songs.
Row 0 - Column recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-2: Found 9 valid songs.
Row 0 - Column recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-zero-shot: Found 7 valid songs.
Row 0 - Column recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-ICL-Few-shot-1: Found 6 valid songs.
Row 0 - Column recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-ICL-Few-shot-2: Found 9 valid songs.
Row 0 - Column recommendation_for_prompt_counterfact-False_sample-random_userDemo-age-group_interact-zero-shot: Found 3 valid songs.
Row 0 - Column recommenda

In [ ]:
import pandas as pd

# data = {
#     'userId': [6, 46, 48],
#     'gender': ['Female', 'Female', 'Male'],
#     'age_group': ['Early Adult (≤24 yrs)', 'Early Adult (≤24 yrs)', 'Early Adult (≤24 yrs)'],
#     'itemIds': [['3', '1', '2', '3'], ['13', '74', '77', '155'], ['357', '1706', '1707', '1708']],
#     'recommendation_for_prompt_1': [[0, 99], [13, 999], [1962, 1706]],
#     'recommendation_for_prompt_2': [[1, 3], [74, 155], [357, 1707]],
#     'recommendation_for_prompt_3': [[3], '77', 1708],
# }

# final_df = pd.DataFrame(data)

def normalize_recommendations(rec):
    if not isinstance(rec, list):
        return [[rec]]
    return [r if isinstance(r, list) else [r] for r in rec]

def calculate_hit_ratios(df, item_col, recommendation_cols):
    hr1_data = {}
    hr_all_data = {}
    for rec_col in recommendation_cols:
        df[rec_col] = df[rec_col].apply(normalize_recommendations)

        hr1_data[f'HR@1_{rec_col}'] = df.apply(
            lambda x: int(str(x[rec_col][0][0]) == str(x[item_col][0]))
            if x[rec_col][0] and x[item_col] and str(x[rec_col][0][0]) != '-1' and str(x[item_col][0]) != '-1'
            else 0,
            axis=1
        )

        hr_all_data[f'HR@all_{rec_col}'] = df.apply(
            lambda x: sum(str(item) in map(str, x[item_col]) for item in x[rec_col][0]) / len(x[rec_col][0])
            if x[rec_col][0] and x[item_col]
            else 0,
            axis=1
        )

    hr1_df = pd.DataFrame(hr1_data)
    hr_all_df = pd.DataFrame(hr_all_data)

    return pd.concat([df, hr1_df, hr_all_df], axis=1)

recommendation_cols = [col for col in final_df.columns if col.endswith('_itemIds')]
final_eval_df= calculate_hit_ratios(final_df, 'itemIds', recommendation_cols)

print(final_eval_df)



IndexError: list index out of range

In [ ]:

# Filtering columns that start with 'recommendation'
recommendation_cols = [col for col in final_eval_df.columns if col.startswith('HR@')]

# Calculating the average for each recommendation column
averages = final_eval_df[recommendation_cols].mean()

print("Averages of Recommendation Columns:")
print(averages)

Averages of Recommendation Columns:
HR@1_recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-zero-shot                                  0.0
HR@1_recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-1                             0.0
HR@1_recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-2                             0.0
HR@1_recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-zero-shot                                   0.0
HR@1_recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-ICL-Few-shot-1                              0.0
                                                                                                                                   ... 
HR@all_recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-age-group_interact-ICL-Few-shot-1_itemIds         0.0
HR@all_recom

In [ ]:

# Calculate statistics from these averages
stats = {
    'mean': averages.mean(),
    'min': averages.min(),
    'max': averages.max(),
    'non_zero_count': averages[averages != 0].count(),
    'non_zero_proportion': 100*averages[averages != 0].count() / len(averages)  # Non-zero proportion
}

print("Statistics derived from Averages of HR@ Recommendation Columns:")
print(stats)

Statistics derived from Averages of HR@ Recommendation Columns:
{'mean': 0.00013020833333333336, 'min': 0.0, 'max': 0.0125, 'non_zero_count': 3, 'non_zero_proportion': 1.0416666666666667}


In [ ]:
# Identifying columns that contain track ID data
track_id_columns = [col for col in final_df.columns if col.startswith("recommendation")]

# Counting non-10000 entries across these specific columns
non_default_id_counts = (final_df[track_id_columns] != -1).sum().sum()
print(f"Number of track ID entries not equal to -1: {non_default_id_counts}")


Number of track ID entries not equal to -1: 11520


In [ ]:
RecLLM_parsed_df

,userId,gender,age_group,itemIds,artistIds,recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-zero-shot,recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-False_sample-random_userDemo-no_info_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-zero-shot,recommendation_for_prompt_counterfact-False_sample-random_userDemo-gender_interact-ICL-Few-shot-1,...,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-no_info_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-gender_interact-zero-shot,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-gender_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-gender_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-age-group_interact-zero-shot,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-age-group_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-age-group_interact-ICL-Few-shot-2,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-intersectional_interact-zero-shot,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-intersectional_interact-ICL-Few-shot-1,recommendation_for_prompt_counterfact-True_sample-recent-frequent_userDemo-intersectional_interact-ICL-Few-shot-2
0,6,Female,Early Adult (≤24 yrs),"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[813, 8857, 6855, 7218, 2643, 7105, 7728, 4990...",56695,35101,-1,-1,9,...,-1,469,465,80893,469,465,-1,469,465,80893
1,46,Female,Early Adult (≤24 yrs),"[13, 74, 77, 155, 195, 203, 204, 217, 341, 345...","[4990, 2187, 6752, 4270, 2247, 2599, 3399, 502...",14803,14803,1579,1705,14803,...,6119,1689,-1,39950,1689,45058,6119,1689,-1,8624
2,48,Male,Early Adult (≤24 yrs),"[357, 1706, 1707, 1708, 1709, 1710, 1711, 1712...","[5237, 7143, 23731, 9565, 22461, 224, 18603, 3...",1962,-1,-1,1962,41696,...,1954,1956,41695,75914,1956,1956,75914,1956,41695,52274
3,57,Female,Early Adult (≤24 yrs),"[654, 787, 790, 792, 798, 891, 938, 1340, 1768...","[2133, 4465, 4286, 3366, 2157, 1506, 4176, 222...",24289,2844,31780,3049,17982,...,3036,1882,2283,3040,1882,2283,2283,2961,1768,2283
4,65,Female,Early Adult (≤24 yrs),"[3050, 3051, 3052, 3053, 3054, 3055, 3056, 305...","[30582, 16460, 27249, 9635, 16458, 30841, 2797...",3290,80019,-1,3239,3167,...,3094,3094,3094,3094,3094,3094,3094,3094,3094,3094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,939,Male,Early Adult (≤24 yrs),"[581, 640, 815, 818, 819, 820, 1535, 1789, 179...","[1109, 1972, 3214, 2879, 2157, 1904, 3307, 296...",53598,23676,-1,53598,23676,...,75533,75539,75533,75533,17455,75533,75533,17455,75533,75533
76,957,Male,Early Adult (≤24 yrs),"[617, 679, 1354, 1355, 1356, 1357, 1397, 1434,...","[1697, 2852, 2241, 2625, 4182, 2927, 3307, 467...",43477,-1,43477,9663,30256,...,22478,42637,73884,43477,9037,-1,-1,42637,9440,43477
77,971,Female,Early Adult (≤24 yrs),"[171, 462, 593, 631, 818, 1178, 1179, 1337, 14...","[1983, 4397, 3222, 2208, 1109, 122, 4286, 578,...",50968,19986,20913,65335,19986,...,20913,77036,-1,2909,-1,1789,622,-1,20913,622
78,980,Female,Early Adult (≤24 yrs),"[297, 506, 507, 564, 568, 572, 593, 643, 646, ...","[3828, 4542, 1721, 2413, 3222, 2133, 3419, 255...",1254,5219,5219,1254,5219,...,21428,32164,2532,21428,21428,21428,5219,32164,39451,21428
